In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点 AI 模型监控和可解释 AI 特征归因

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/model_monitoring.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"> 在 Vertex AI Workbench 中打开
    </a>
</table>

## 概述

### 什么是 Vertex AI 模型监控？

现代应用程序依赖于一套完善的能力来监视其服务的健康状况。示例包括：

* 软件版本控制
* 严格的部署流程
* 事件记录
* 警报/通知需要干预的情况
* 需求和自动诊断跟踪
* 自动化性能和功能测试

您应该能够像管理应用程序一样强大和灵活地管理您的 ML 服务。这就是 MLOps 的核心 - 通过 Google 和更广泛的计算行业从部署经过良好设计、可靠和可扩展服务的多代经验中学到的最佳实践来管理 ML 服务。

模型监控只是 ML Ops 谜题中的一部分 - 它有助于回答以下问题：

* 最近的服务请求与用于构建模型的训练数据匹配得有多好？这被称为 **训练-服务偏差**。
* 服务请求随时间的演变有多显著？这被称为 **漂移检测**。

[Vertex 可解释 AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview) 在模型监控中增加了另一个方面，我们称之为特征归属监控。可解释 AI 使您能够了解每个特征对结果预测的相对贡献。实质上，它评估了每个特征影响的幅度。

如果生产流量与训练数据不同，或随时间变化较大，**无论是模型预测还是特征归因方面**，这可能会影响您的模型提供的答案质量。当发生这种情况时，您希望能够自动和响应地收到警报，以便 **在问题影响客户体验或收入流之前预见问题**。

了解更多关于 [Vertex AI 模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring)。

### 目标

在这个笔记本中，您将学习如何使用`Vertex AI Model Monitoring`服务来检测部署的`Vertex AI Model`资源中预测请求的漂移和异常。

本教程使用以下谷歌云机器学习服务：

- `Vertex AI Model Monitoring`
- `Vertex AI Prediction`
- `Vertex AI Model`资源
- `Vertex AI Endpoint`资源

执行的步骤包括：

- 将预训练模型上传为`Vertex AI Model`资源。
- 创建一个`Vertex AI Endpoint`资源。
- 将`Model`资源部署到`Endpoint`资源。
- 配置`Endpoint`资源以进行模型监控。
- 初始化模型监控的基线分布。
- 生成合成预测请求。
- 了解如何解释模型监控功能报告的统计数据、可视化和其他数据。

### 模型

本教程使用一个预先训练的模型，模型的构件存储在一个公共的云存储桶中。该模型用于预测在线游戏网站中玩家可能流失的概率，即停止成为活跃玩家。

成本

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- BigQuery
- Cloud Storage

了解 [Vertext AI 定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 定价](https://cloud.google.com/storage/pricing) 和 [BigQuery 定价](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成一个成本估算。

安装

安装执行此笔记本所需的软件包。

In [ ]:
# Don't bother installing tensorflow or explainable_ai_sdk on Colab

# Install required packages.
! pip3 install --upgrade --quiet \
    google-cloud-aiplatform \
    google-cloud-bigquery \
    explainable_ai_sdk 
#! pip3 install --upgrade --quiet tensorflow==2.11
! pip3 install --force-reinstall protobuf==4.23.3

只有协作：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"

用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面相应的说明。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经通过验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上给您的服务账号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 关于服务账户和权限的注意事项

** 默认情况下无需配置**，如果遇到任何与权限相关的问题，请确保上述服务账户具有所需的角色：

| 服务账户电子邮件 | 描述 | 角色 |
| --- | --- | --- |
| PROJECT_NUMBER-compute@developer.gserviceaccount.com | 计算引擎默认服务账户 | Dataflow Admin, Dataflow Worker, Storage Admin, BigQuery Admin, Vertex AI User |
| service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com | AI平台服务代理 | Vertex AI服务代理 |

1. 访问 https://console.cloud.google.com/iam-admin/iam。
2. 勾选“包括由Google提供的角色授予”复选框。
3. 找到上述电子邮件。
4. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请为两个服务账户授予“BigQuery Data Viewer”角色。
- 对于CSV数据源，请为两个服务账户授予“Storage Object Viewer”角色。

导入库并定义常量

In [ ]:
# Import required packages.
import os
import pprint as pp

import google.cloud.aiplatform as aiplatform
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

In [ ]:
if os.getenv("IS_TESTING"):
    ! gcloud --quiet components install beta
    ! gcloud --quiet components update

! gcloud config set ai/region $REGION
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

### 初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### 创建BigQuery客户端

在本教程中，您将使用与训练预训练模型相同的公共BigQuery表中的数据。您将创建一个客户端接口，随后用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### 示例模型

您在本笔记本中使用的模型是基于[此博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)的。这个模型的理念是，您的公司拥有描述游戏用户与网站互动情况的大量日志数据。原始数据包含以下信息类别：

- 身份 - 独特的玩家身份编号
- 人口统计特征 - 有关玩家的信息，例如玩家所在地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，例如达到新的等级
- 流失倾向 - 这是标签或目标特征，它提供了该玩家流失，即不再是活跃玩家的估计概率。

上面引用的博客文章解释了如何使用BigQuery存储原始数据，为机器学习预处理数据，并训练相应的模型。因为这个笔记本侧重于模型监控，而不是训练模型，您将重复使用已经导出到云存储的预训练版本的此模型。在下一部分，您将设置环境并将此模型导入到您自己的项目中。

### 定义一些辅助的数据结构

运行以下单元格以定义在整个笔记本中使用的一些数据结构。

In [ ]:
# @title Utility data structures

# Sampling distributions for categorical features...
DAYOFWEEK = {1: 1040, 2: 1223, 3: 1352, 4: 1217, 5: 1078, 6: 1011, 7: 1110}

LANGUAGE = {
    "en-us": 4807,
    "en-gb": 678,
    "ja-jp": 419,
    "en-au": 310,
    "en-ca": 299,
    "de-de": 147,
    "en-in": 130,
    "en": 127,
    "fr-fr": 94,
    "pt-br": 81,
    "es-us": 65,
    "zh-tw": 64,
    "zh-hans-cn": 55,
    "es-mx": 53,
    "nl-nl": 37,
    "fr-ca": 34,
    "en-za": 29,
    "vi-vn": 29,
    "en-nz": 29,
    "es-es": 25,
}

OS = {"IOS": 3980, "ANDROID": 3798, "null": 253}

MONTH = {6: 3125, 7: 1838, 8: 1276, 9: 1718, 10: 74}

COUNTRY = {
    "United States": 4395,
    "India": 486,
    "Japan": 450,
    "Canada": 354,
    "Australia": 327,
    "United Kingdom": 303,
    "Germany": 144,
    "Mexico": 102,
    "France": 97,
    "Brazil": 93,
    "Taiwan": 72,
    "China": 65,
    "Saudi Arabia": 49,
    "Pakistan": 48,
    "Egypt": 46,
    "Netherlands": 45,
    "Vietnam": 42,
    "Philippines": 39,
    "South Africa": 38,
}

# Means and standard deviations for numerical features...
MEAN_SD = {
    "julianday": (204.6, 34.7),
    "cnt_user_engagement": (30.8, 53.2),
    "cnt_level_start_quickplay": (7.8, 28.9),
    "cnt_level_end_quickplay": (5.0, 16.4),
    "cnt_level_complete_quickplay": (2.1, 9.9),
    "cnt_level_reset_quickplay": (2.0, 19.6),
    "cnt_post_score": (4.9, 13.8),
    "cnt_spend_virtual_currency": (0.4, 1.8),
    "cnt_ad_reward": (0.1, 0.6),
    "cnt_challenge_a_friend": (0.0, 0.3),
    "cnt_completed_5_levels": (0.1, 0.4),
    "cnt_use_extra_steps": (0.4, 1.7),
}

DEFAULT_INPUT = {
    "cnt_ad_reward": 0,
    "cnt_challenge_a_friend": 0,
    "cnt_completed_5_levels": 1,
    "cnt_level_complete_quickplay": 3,
    "cnt_level_end_quickplay": 5,
    "cnt_level_reset_quickplay": 2,
    "cnt_level_start_quickplay": 6,
    "cnt_post_score": 34,
    "cnt_spend_virtual_currency": 0,
    "cnt_use_extra_steps": 0,
    "cnt_user_engagement": 120,
    "country": "Denmark",
    "dayofweek": 3,
    "julianday": 254,
    "language": "da-dk",
    "month": 9,
    "operating_system": "IOS",
    "user_pseudo_id": "104B0770BAE16E8B53DF330C95881893",
}

### 为Vertex可解释人工智能生成模型元数据

运行以下单元格，从导出的模型中提取元数据，这些元数据在生成预测请求的解释时是必要的。

In [ ]:
MODEL_PATH = "gs://mco-mm/churn"

params = {"sampled_shapley_attribution": {"path_count": 10}}
EXPLAIN_PARAMS = aiplatform.explain.ExplanationParameters(params)

builder = saved_model_metadata_builder.SavedModelMetadataBuilder(
    model_path=MODEL_PATH, outputs_to_explain=["churned_probs"]
)
EXPLAIN_META = builder.get_metadata_protobuf()

上传您的模型

您在此笔记本中使用的流失倾向模型是在BigQuery ML中训练并导出到Cloud Storage存储桶中的。这展示了您如何轻松地导出一个经过训练的模型，并将模型从一个云服务移动到另一个云服务。

运行下一个单元格以将此模型导入到您的项目中。**如果您已经导入了您的模型，则可以跳过此步骤。**

In [ ]:
MODEL_NAME = "churn"
IMAGE = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-5:latest"

model = aiplatform.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=MODEL_PATH,
    serving_container_image_uri=IMAGE,
    explanation_parameters=EXPLAIN_PARAMS,
    explanation_metadata=EXPLAIN_META,
    sync=True,
)

MODEL_ID = model.resource_name.split("/")[-1]

当上述单元格完成后，您应该在云控制台的Vertex AI模型注册表页面上看到一个新模型。

## 部署您的模型资源到终端资源

现在您已经将模型导入到项目中，您需要创建一个终端点来提供模型服务。终端点可以被视为模型提供预测服务的通道。一旦建立，您可以通过公共互联网对模型进行在线预测请求。您的终端点也是无服务器的，谷歌云通过减少单点故障并动态分配资源以满足服务需求来确保高可用性和可扩展性。通过这种方式，您可以专注于您的模型质量，并摆脱了管理和基础设施方面的顾虑。

运行下一个单元格将您的模型部署到终端点。**这将大约需要十分钟完成。**

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")
print(f"endpoint display name: {endpoint.display_name}")
print(f"endpoint resource name: {endpoint.resource_name}")
ENDPOINT = endpoint.resource_name
ENDPOINT_ID = ENDPOINT.split("/")[-1]

当上述单元格完成后，您应该在Cloud控制台的Vertex AI端点页面上看到一个新的端点。

运行一个预测测试

现在你已经导入了一个模型并将该模型部署到一个端点，你已准备好验证它是否运行正常。运行下一个单元格发送一个测试预测请求。如果一切正常，你将收到一个文本表示为JSON编码的响应，还有一个总结结果的饼图。

**现在尝试通过运行下一个单元格来实现。**

In [ ]:
try:
    resp = endpoint.predict([DEFAULT_INPUT])
    for i in resp.predictions:
        vals = i["churned_values"]
        probs = i["churned_probs"]
    for i in range(len(vals)):
        print(vals[i], probs[i])
    plt.pie(probs, labels=vals)
    pp.pprint(resp)
except Exception as ex:
    print("prediction request failed", ex)

### 测试结果

查看结果，您会看到以下元素：

- **churned_values** - 目标字段可能的值集合（0和1）
- **churned_probs** - 每个可能目标字段值对应的概率集合（分别为5x10^-40和1.0）
- **predicted_churn** - 基于概率，目标字段的预测值（1）

此响应以软件易于理解的格式对模型的预测进行编码，使得这项服务非常适合应用程序的自动化使用。

运行解释性测试

您可以在此端点上运行可解释的人工智能测试。运行下一个单元格以发送测试解释请求。您收到的响应将以称为JSON的文本表示形式编码这次预测的特征重要性，同时还包括一个总结结果的条形图。

**现在通过运行下一个单元格来尝试。**

In [ ]:
try:
    features = []
    scores = []
    resp = endpoint.explain([DEFAULT_INPUT])
    for i in resp.explanations:
        for j in i.attributions:
            for k in j.feature_attributions:
                features.append(k)
                scores.append(j.feature_attributions[k])
    features = [x for _, x in sorted(zip(scores, features))]
    scores = sorted(scores)
    fig, ax = plt.subplots()
    fig.set_size_inches(9, 9)
    ax.barh(features, scores)
    fig.show()
except Exception as ex:
    print("explanation request failed", ex)

##开始您的监控工作

现在您已经创建了一个用于为您的模型提供预测请求的端点，您可以开始一个监控工作，以监测模型质量并在输入开始偏离可能影响模型预测质量的方式时提醒您。

在这一部分中，您将根据您从BigQuery ML导入的流失倾向模型配置和创建一个模型监控作业。

### 配置以下字段：

1. 日志采样率 - 您的预测请求和响应将记录到BigQuery表中，在创建监控作业时会自动生成这些表。该参数指定了这些表的所需记录频率。
1. 监控间隔 - 分析数据和报告异常的时间窗口。最小窗口为一小时（1）
1. 目标字段 - 训练数据集中的预测目标列名
1. 偏斜检测阈值 - 您想监视的每个特征的偏斜阈值
1. 预测漂移阈值 - 您想监视的每个特征的漂移阈值
1. 归因偏斜检测阈值 - 特征重要性的偏斜阈值
1. 归因预测漂移阈值 - 特征重要性的漂移阈值

In [ ]:
JOB_NAME = "churn"

# Sampling rate (optional, default=.8)
LOG_SAMPLE_RATE = 0.8  # @param {type:"number"}

# Monitoring Interval in hours (optional, default=1).
MONITOR_INTERVAL = 1  # @param {type:"number"}

# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

# # Skew and drift thresholds.

DEFAULT_THRESHOLD_VALUE = 0.001

SKEW_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}
DRIFT_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}
ATTRIB_SKEW_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}
ATTRIB_DRIFT_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}

您可以更改阈值和配置设置，以便您还可以监视模型中的其他特征。

创建您的监控任务

以下代码使用Google Python客户端库将您的配置设置转换为一个编程请求，以启动一个模型监控任务。实例化一个监控任务可能需要一些时间。如果您的请求看起来一切正常，您将收到一个成功的API响应。然后，您需要检查您的电子邮件，以接收通知该任务正在运行。

In [ ]:
skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI,
    skew_thresholds=SKEW_THRESHOLDS,
    attribute_skew_thresholds=ATTRIB_SKEW_THRESHOLDS,
    target_field=TARGET,
)

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=DRIFT_THRESHOLDS,
    attribute_drift_thresholds=ATTRIB_DRIFT_THRESHOLDS,
)

explanation_config = model_monitoring.ExplanationConfig()
objective_config = model_monitoring.ObjectiveConfig(
    skew_config, drift_config, explanation_config
)

# Create sampling configuration
random_sampling = model_monitoring.RandomSampleConfig(sample_rate=LOG_SAMPLE_RATE)

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

# Create alerting configuration.
emails = [USER_EMAIL]
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=emails, enable_logging=True
)

# Create the monitoring job.
job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name=JOB_NAME,
    logging_sampling_strategy=random_sampling,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
)

### 接收电子邮件提醒
一分钟或两分钟后，您应该会收到发送到您设置的USER_EMAIL地址的电子邮件。这封邮件确认了您的监控作业部署成功。以下是这封电子邮件可能的样本：
<br>
<br>
<img src="https://storage.googleapis.com/mco-general/img/mm6.png" />
<br>
随着您的监控作业收集数据，测量数据将存储在Cloud Storage中，您可以随时查看您的数据。"统计和异常根路径"指定了Cloud Storage中测量数据的位置。运行以下单元格以查看这些测量数据在Cloud Storage中的布局示例。如果您在作业创建电子邮件中替换Cloud Storage URL，您可以查看您自己的监控作业的数据文件的结构和内容。

创造抽样分布

接下来，您发送第一个测试预测请求。模型监控服务将分析特征的分布并自动创建一个基准线以监测与基准线的偏差。

*注：*在进行第一个预测请求之前，您需要等待电子邮件通知。

## 初始化解析以自动生成输入模式

当您的`Endpoint`收到1000个预测请求后，建模服务将自动解析并创建`输入模式`。

### 创建1000个实例数据

在这个例子中，BigQuery训练数据中的前1000个条目被用作前1000个预测请求。

*注意:* 在这个上下文中，每个实例都是一个预测请求。换句话说，发送1000个相同实例的预测请求与发送一个具有1000个实例的预测请求是相同的。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发起初始预测请求

接下来，您可以使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
response = endpoint.predict(instances=instances)

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 输入模式的自动生成

在模型监控服务接收到1000个预测请求实例后，监控将开始分析这些预测请求，以自动生成一个用于特征输入的`输入模式`。

### 基线分布的自动生成

在生成了`输入模式`之后，监控服务创建一个批处理作业来分析训练数据，以确定基线分布。

In [ ]:
# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    import time

    time.sleep(120)

### 云存储中存储的监控数据示例

In [ ]:
!gsutil ls gs://cloud-ai-platform-fdfb4810-148b-4c86-903c-dbdff879f6e1/*/*

### 云存储布局
请注意以下组件在这些云存储路径中：

- **cloud-ai-platform-..** - 这是为您创建的一个存储桶，并指定用于捕获您服务的预测数据。您创建的每个监控作业都会触发在此存储桶中创建一个新文件夹。
- **[model_monitoring|instance_schemas]/job-..** - 这是您独特的监控作业编号，您可以在作业创建请求的响应和电子邮件通知中看到。
- **instance_schemas/job-../analysis** - 这是监控作业了解和编码您训练数据模式（字段名称、类型等）的过程。
- **instance_schemas/job-../predict** - 这是当前监控作业启用后对您模型进行的第一次预测。
- **model_monitoring/job-../serving** - 这个文件夹用于记录漂移计算相关的数据。它包含了每小时模型提供流量的测量摘要。
- **model_monitoring/job-../training** - 这个文件夹用于记录与训练-提供偏移计算相关的数据。它包含了预测数据相对于训练数据的持续摘要。
- **model_monitoring/job-../feature_attribution_score** - 这个文件夹用于记录与特征归因计算相关的数据。它包含了特征归因分数相对于训练数据的持续摘要。

您可以使用其他用户界面创建监控任务

在之前的单元格中，您使用Python客户端库创建了一个监控任务。或者，您可以使用*gcloud*命令行工具或云控制台来创建模型监视任务。

解释您的结果

当超过为特征设置的阈值时，Vertex AI模型监控会检测到异常。下面的单元格可以让您了解在检测到模型监控异常后的警报和报告体验。

Vertex AI模型监控会通过电子邮件自动通知您检测到的异常，但您也可以通过[Cloud Logging设置警报](https://cloud.google.com/vertex-ai/docs/model-monitoring/using-model-monitoring#monitor-job)。

以下是示例电子邮件提醒的样式...

<img src="https://storage.googleapis.com/mco-general/img/mm7.png" />

这封邮件是在警告您，生产中看到的 *cnt_level_start_quickplay*、*cnt_user_engagement* 和 *country* 特征值偏离了您在训练和服务您的模型之间设定的阈值。它还告诉您，*cnt_user_engagement* 和 *country* 特征归因值相对于您的训练数据也有偏差，再次根据您的阈值设定。

### 在云控制台监控结果

您可以从云控制台检查您的模型监控数据。以下是这些功能的截图。

#### 监控状态

您可以通过端点摘要页面验证特定端点是否有活动的模型监控作业：

<img src="https://storage.googleapis.com/mco-general/img/mm1.png" />

#### 监控警报

您可以通过单击感兴趣的端点并选择警报面板来查看警报详细信息：

<img src="https://storage.googleapis.com/mco-general/img/mm2.png" />

#### 特征值分布

您还可以通过深入研究特定特征来检查记录的训练和生产特征分布，例如：

<img src="https://storage.googleapis.com/mco-general/img/mm9.png" />

这会在训练和生产期间获得特征分布的图形表示，例如：

<img src="https://storage.googleapis.com/mco-general/img/mm8.png" />

## 清理

当您完成这个笔记本时，您可以清理这个项目中使用的所有谷歌云资源，方法是删除您用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以保留该项目，并通过执行以下单元格中的操作删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model and delete the endpoint
endpoint.undeploy_all()
endpoint.delete()

model.delete()

# Delete BQ table and dataset
rmtable = f"bq rm -f model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
! $rmtable
rmdataset = f"bq rm -f model_deployment_monitoring_{ENDPOINT_ID}"
! $rmdataset

学习更多关于模型监控

**恭喜！** 您已经了解了模型监控是什么，如何配置和启用它，以及如何查找和解释结果。查看以下资源，了解更多关于模型监控和ML Ops 的信息。

- [TensorFlow 数据验证](https://www.tensorflow.org/tfx/guide/tfdv)
- [规模化的数据理解、验证和监控](https://blog.tensorflow.org/2018/09/introducing-tensorflow-data-validation.html)
- [Vertex 产品文档](https://cloud.google.com/vertex-ai)
- [Vertex AI 模型监控参考文档](https://cloud.google.com/vertex-ai/docs/reference)
- [Vertex AI 模型监控博客文章](https://cloud.google.com/blog/topics/developers-practitioners/monitor-models-training-serving-skew-vertex-ai)
- [可解释的AI白皮书](https://storage.googleapis.com/cloud-ai-whitepapers/AI%20Explainability%20Whitepaper.pdf)